In [2]:
import re, os, glob
import pandas as pd
import numpy as np

from collections import defaultdict

$$ \textbf{(1)} \space \space D = \sum_{i=1}^{R} \left( \frac{n_i}{N}\right)^{\!2} $$
$$ \textbf{(2)} \space \space D = \sum_{i=1}^{R} \frac{n_i(n_i-1)}{N(N-1)} $$

$$ \textbf{(3)} \space \space D_{comp} = 1-D $$
$$ \textbf{(4)} \space \space D_{inv} = \frac{1}{D} $$

# Functions

In [10]:
def read_file(file):
    """
    This function reads in the input file and counts the number of UMIs for each TCR.
    """
    tcr_dict = defaultdict(int)
    # Read in Fixed Cut Files
    with open(file) as input:
        for line in input:
            info = line.rstrip().split('\t')
            tcr_aa = '|'.join(info[:3])
            umi = int(info[3])
            tcr_dict[tcr_aa] += umi
    return tcr_dict

In [21]:
def extract_donor_info(filename):
    match = re.search(r"P7M1S-(.*)-(\d{3,4})([AB])CD(4|8|48)(N|M|NM)-(alpha|beta)combinedfixed_cut.txt",
              os.path.basename(filename))
    if match:
        donor,timepoint,cell_type,chain = match.group(2),match.group(3),'CD{}{}'.format(
            match.group(4),match.group(5)),match.group(6)        
    else:
        print("Error! Filename {} does not match convention!".format(filename))
        sys.exit(1)
    return donor,timepoint, cell_type,chain        

In [31]:
def calculate_simpson(tcr_df):
    total_umi = float(tcr_df['UMI Count'].sum())
    tcr_df['UMI Abundance'] = (tcr_df['UMI Count']/total_umi)**2
    tcr_umi_df['UMI Adjustment'] = tcr_umi_df['UMI Count'] * (tcr_umi_df['UMI Count'] - 1)
    
    simpson_1 = tcr_df['UMI Abundance'].sum()
    simpson_2 =  tcr_umi_df['UMI Adjustment'].sum()/(total_umi*(total_umi-1))
    comp_sim1 = 1-simpson_1
    comp_sim2 = 1-simpson_2
    inv_sim1 = 1/simpson_1
    inv_sim2 = 1/simpson_2
    
    return simpson_1, simpson_2, comp_sim1, comp_sim2, inv_sim1, inv_sim2

# Read Input Files

In [38]:
input_dir = 'C:/Users/super/OneDrive - University of California, Davis/NIH/Human_TCR_Analysis/Data_Files/CDR3_AA/CD4_CD8_redo/'
input_files = sorted(glob.glob(input_dir + '*fixed_cut.txt'))
print(len(input_files))

240


In [39]:
summary_df = pd.DataFrame(index = range(0,len(input_files)),
                         columns = ['Full Sample Name','Donor','Timepoint','Cell Type','TCR Chain','Total TCR Count',
                                    'Total UMI Count','Simpson Index-1','Simpson Index-2',
                                    'Complementary Simpson Index-1','Complementary Simpson Index-2',
                                    'Inverse Simpson Index-1','Inverse Simpson Index-2'])

# Calculate Simpson's Index

In [40]:
counter = -1
for file in input_files:
    counter += 1
    tcr_umi_dict = read_file(file)
    donor,timepoint, cell_type,chain = extract_donor_info(file)
    full_samp_name = '{}{}{}-{}'.format(donor,timepoint, cell_type,chain)
    tcr_umi_df = pd.DataFrame.from_dict(tcr_umi_dict,orient = 'index',columns = ['UMI Count'])
    total_tcr_count = len(tcr_umi_df.index.values)
    total_umi_count = tcr_umi_df['UMI Count'].sum()
    s1,s2,cs1,cs2,is1,is2 = calculate_simpson(tcr_umi_df)
    
    summary_df.loc[counter,'Full Sample Name'] = full_samp_name
    summary_df.loc[counter,'Donor'] = donor
    summary_df.loc[counter,'Timepoint'] = timepoint
    summary_df.loc[counter,'Cell Type'] = cell_type
    summary_df.loc[counter,'TCR Chain'] = chain
    summary_df.loc[counter,'Total TCR Count'] = total_tcr_count
    summary_df.loc[counter,'Total UMI Count'] = total_umi_count
    summary_df.loc[counter,'Simpson Index-1'] = s1
    summary_df.loc[counter,'Simpson Index-2'] = s2
    summary_df.loc[counter,'Complementary Simpson Index-1'] = cs1
    summary_df.loc[counter,'Complementary Simpson Index-2'] = cs2
    summary_df.loc[counter,'Inverse Simpson Index-1'] = is1
    summary_df.loc[counter,'Inverse Simpson Index-2'] = is2
    
    

In [41]:
summary_df.head()

,Full Sample Name,Donor,Timepoint,Cell Type,TCR Chain,Total TCR Count,Total UMI Count,Simpson Index-1,Simpson Index-2,Complementary Simpson Index-1,Complementary Simpson Index-2,Inverse Simpson Index-1,Inverse Simpson Index-2
0,1029ACD4NM-alpha,1029,A,CD4NM,alpha,28061,568468,0.00194663,0.00194488,0.998053,0.998055,513.707,514.171
1,1029ACD4NM-beta,1029,A,CD4NM,beta,27427,636156,0.000122587,0.000121015,0.999877,0.999879,8157.47,8263.42
2,1029ACD8NM-alpha,1029,A,CD8NM,alpha,6488,351306,0.0194444,0.0194416,0.980556,0.980558,51.4287,51.4361
3,1029ACD8NM-beta,1029,A,CD8NM,beta,9680,992546,0.0416692,0.0416682,0.958331,0.958332,23.9985,23.9991
4,1029BCD4NM-alpha,1029,B,CD4NM,alpha,19239,404551,0.000500008,0.000497537,0.9995,0.999502,1999.97,2009.9


In [42]:
summary_df.to_csv('./A_Results/simpson_index_results-cd4_cd8.csv')